In [1]:
pathtorepo = "C:\\Users\\Alex\\Desktop\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [2]:
using StaticArrays, DifferentialEquations, DynamicalSystems, ForwardDiff
E, x, y  = 0..20, 0..20,  0..20
box = E × x × y
using CairoMakie, GLMakie;

In [3]:
function RMA(u, p, t)
    x,y,z = u
    r, k, a1, b1, a2, b2, d1, d2 = p
    
   ẋ = x * ( r * ( 1 - x/k ) - a1 * y / ( 1 + b1 * x ) )
   ẏ = y * ( a1 * x / ( 1 + b1 * x ) - a2 * z / ( 1 + b2 * y ) - d1 )
   ż = z * ( a2 * y / ( 1 + b2 * y ) - d2 )
    return SVector(ẋ , ẏ, ż)
end

RMA (generic function with 1 method)

In [4]:
function jac_RMA(u, p, t)
    x,y,z = u
    r, k, a1, b1, a2, b2, d1, d2 = p

    fx = x * ( -r/k + a1 *y * b1 / (b1 * x + 1)^2 ) + r * (1 - x/k) - a1*y/(1+b1*x) 
    gy = -a2 * z / ( b2 * y + 1 ) + a2 * b2 * z * y / ( b2 * y + 1 )^2 + a1 * x / (b1 * x + 1) - d1
    hz = a2 * y / ( 1 + b2 * y ) - d2

    fy = -a1 * x / (1 + b1 * x)
    gz = -a2 * y / (1 + b2 * y)

    gx = y * a1 / ( b1 * x + 1 )^2
    hy = z * a2 / ( b2 * y + 1 )^2

    return SMatrix{3, 3}( fx, gx, 0.0,
                    fy, gy, hy,
                    0.0, gz, hz )
end

jac_RMA (generic function with 1 method)

In [5]:
a1 = 5; a2 = 0.1; b1 = 3; b2 = 2; d1 = 0.4; d2 = 0.01

0.01

In [198]:
r = 0.8122
k = 1.08
p = [r, k, a1, b1, a2, b2, d1, d2]

8-element Vector{Float64}:
 0.8122
 1.08
 5.0
 3.0
 0.1
 2.0
 0.4
 0.01

In [199]:
u0 = [0.7297732151428117, 0.1889267215901415, 9.585811098760479]

3-element Vector{Float64}:
 0.7297732151428117
 0.1889267215901415
 9.585811098760479

In [200]:
t = 5000; tt = 10000;

In [201]:
Jacobi(u0) = ForwardDiff.jacobian((x) -> RMA(x, p, t), u0)
ds = CoupledODEs(RMA, shift, p, diffeq = integrator_setting)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  RMA
 ODE solver:    RK4
 ODE kwargs:    (adaptive = false, dt = 0.01)
 parameters:    [0.8122, 1.08, 5.0, 3.0, 0.1, 2.0, 0.4, 0.01]
 time:          0.0
 state:         [0.8628008036745088, 0.12499999219651951, 10.027592329006737]


In [202]:
fp, eigs, _ = fixedpoints(ds, box, jac_RMA)

┌ Warning: Non-unique fixed points found!
└ @ ChaosTools C:\Users\Alex\.julia\packages\ChaosTools\VMXUY\src\stability\fixedpoints.jl:104


(3-dimensional StateSpaceSet{Float64} with 4 points, Vector{ComplexF64}[[-0.40554536918683914 + 0.0im, 0.027996008326492603 - 0.08481459656772938im, 0.027996008326492603 + 0.08481459656772938im], [0.003920007024253073 + 0.0im, 0.04838368810916177 - 0.46957707944465293im, 0.04838368810916177 + 0.46957707944465293im], [-0.399999999999999 + 0.0im, -0.009999999999999962 + 0.0im, 0.812199999999998 + 0.0im], [-0.8122 + 0.0im, -0.009999999999999998 + 0.0im, 0.8735849056603774 + 0.0im]], Bool[0, 0, 0, 0])

In [203]:
Jac = jac_RMA(fp[1], p, 0)
eivecs = eigvecs(Jac)
shift = real(eivecs[:, 1])*1e-7 + fp[1]

3-element SVector{3, Float64} with indices SOneTo(3):
 0.844876022477604
 0.12499999148751165
 9.93926622511263

In [204]:
integrator_setting = (alg = RK4(), adaptive = false, dt = 0.01);

In [205]:
fp[2]

3-element SVector{3, Float64} with indices SOneTo(3):
 0.10526315789473684
 0.19290479121781062
 1.5257227967332473e-16

In [211]:
prob = ODEProblem(RMA, shift, (0.0, 1200), p)
sol = solve(prob, alg = RK4(), adaptive = false, dt = 0.001)

retcode: Success
Interpolation: 3rd order Hermite
t: 1200002-element Vector{Float64}:
    0.0
    0.001
    0.002
    0.003
    0.004
    0.005
    0.006
    0.007
    0.008
    0.009000000000000001
    0.010000000000000002
    0.011000000000000003
    0.012000000000000004
    ⋮
 1199.989999978536
 1199.990999978536
 1199.991999978536
 1199.9929999785359
 1199.9939999785358
 1199.9949999785358
 1199.9959999785358
 1199.9969999785358
 1199.9979999785357
 1199.9989999785357
 1199.9999999785357
 1200.0
u: 1200002-element Vector{SVector{3, Float64}}:
 [0.844876022477604, 0.12499999148751165, 9.93926622511263]
 [0.8448760224375692, 0.12499999149096315, 9.939266225107216]
 [0.8448760223975507, 0.12499999149441325, 9.939266225101806]
 [0.8448760223575483, 0.12499999149786195, 9.939266225096397]
 [0.8448760223175622, 0.12499999150130925, 9.93926622509099]
 [0.8448760222775923, 0.12499999150475516, 9.939266225085584]
 [0.8448760222376386, 0.12499999150819967, 9.93926622508018]
 [0.8448760221977

In [212]:
ts, tf = 1, 1050000
GLMakie.activate!()
f = Figure(resolution = (1600, 1200))
ax = Axis3(f[1, 1])
lines!(ax, sol[3, ts:tf], sol[1, ts:tf], sol[2, ts:tf], linewidth = 2.0)
idxfp = 1
#scatter!(ax, fp[idxfp][1], fp[idxfp][2], fp[idxfp][3], color = :deeppink)
display(f)

GLMakie.Screen(...)